In [1]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import string
import re
from bs4 import BeautifulSoup
import requests
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

In [2]:
directory = 'data'
all_data = []
years = []
for file in os.listdir(directory)[3:]:
    filename = file.split('.')[0]
    year = filename.split('-')[-1]
    years.extend([year]*100)
    with open(os.path.join(directory, file), 'r') as f:
        all_data.append(f.name)

all_data

['data\\songs-1970.csv',
 'data\\songs-1971.csv',
 'data\\songs-1972.csv',
 'data\\songs-1973.csv',
 'data\\songs-1974.csv',
 'data\\songs-1975.csv',
 'data\\songs-1976.csv',
 'data\\songs-1977.csv',
 'data\\songs-1978.csv',
 'data\\songs-1979.csv',
 'data\\songs-1980.csv',
 'data\\songs-1981.csv',
 'data\\songs-1982.csv',
 'data\\songs-1983.csv',
 'data\\songs-1984.csv',
 'data\\songs-1985.csv',
 'data\\songs-1986.csv',
 'data\\songs-1987.csv',
 'data\\songs-1988.csv',
 'data\\songs-1989.csv',
 'data\\songs-1990.csv',
 'data\\songs-1991.csv',
 'data\\songs-1992.csv',
 'data\\songs-1993.csv',
 'data\\songs-1994.csv',
 'data\\songs-1995.csv',
 'data\\songs-1996.csv',
 'data\\songs-1998.csv',
 'data\\songs-1999.csv',
 'data\\songs-2000.csv',
 'data\\songs-2001.csv',
 'data\\songs-2002.csv',
 'data\\songs-2003.csv',
 'data\\songs-2005.csv',
 'data\\songs-2006.csv',
 'data\\songs-2007.csv',
 'data\\songs-2008.csv',
 'data\\songs-2009.csv',
 'data\\songs-2010.csv',
 'data\\songs-2011.csv',


In [3]:
df = pd.concat(map(pd.read_csv, all_data), ignore_index=True)
songs_df = df.drop(['Unnamed: 0'], axis=1)
# add the column for year of appearance in the chart
songs_df['year'] = years

print(songs_df.columns)
songs_df

Index(['id', 'title', 'primary_artist', 'all_artists', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'duration_s', 'year'],
      dtype='object')


,id,title,primary_artist,all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_s,year
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,['The Beatles'],0.443,0.403,0,-8.339,1,0.0322,0.6310,0.000000,0.1110,0.4100,143.462,4,243,1970
1,6QhXQOpyYvbpdbyjgAqKdY,Cecilia,Simon & Garfunkel,['Simon & Garfunkel'],0.755,0.876,0,-8.867,1,0.0362,0.3570,0.000005,0.2200,0.9540,102.762,4,175,1970
2,2BhrrVRC8CoiWejbzj0VjQ,Son of a Preacher Man,Aretha Franklin,['Aretha Franklin'],0.474,0.473,2,-11.454,1,0.0601,0.5450,0.000001,0.0356,0.5610,77.583,4,199,1970
3,3Bh6uInhcVBVvLraGZdkKD,Who'll Stop The Rain,Creedence Clearwater Revival,['Creedence Clearwater Revival'],0.598,0.797,7,-6.793,1,0.0332,0.0424,0.000407,0.0717,0.6220,123.566,4,147,1970
4,6UkMcAA19lTdjs22jtB7o2,Big Yellow Taxi,Joni Mitchell,['Joni Mitchell'],0.611,0.470,4,-9.135,1,0.0356,0.5790,0.000000,0.5810,0.9700,85.527,4,135,1970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,7AODNxFNPnKaz4hURLzw5l,Ladbroke Grove,AJ Tracey,['AJ Tracey'],0.902,0.839,11,-9.447,0,0.2080,0.0933,0.000000,0.1020,0.7270,133.989,4,191,2019
4796,44ADyYoY5liaRa3EOAl4uf,Slide Away,Miley Cyrus,['Miley Cyrus'],0.530,0.538,5,-6.672,1,0.0335,0.1700,0.000003,0.2480,0.2880,148.051,4,234,2019
4797,2KKijf6Ett4kmVNJze2roh,The Git Up,Blanco Brown,['Blanco Brown'],0.847,0.678,9,-8.635,1,0.1090,0.0669,0.000000,0.2740,0.8110,97.984,4,201,2019
4798,2bjUEg4jBtKBlPdNrTAppI,Easier,5 Seconds of Summer,['5 Seconds of Summer'],0.562,0.460,5,-4.173,1,0.2590,0.4760,0.000000,0.1070,0.6230,176.055,4,158,2019


In [4]:
## Get genre
load_dotenv()
cid = os.getenv('client_id')
secret = os.getenv('client_secret')

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

chars = re.escape(string.punctuation)
# !"\#\$%\&'\(\)\*\+,\-\./:;<=>\?@\[\\\]\^_`\{\|\}\~
songs_df = songs_df.assign(genres = '')

def get_genre_by_artist(df):
    for artist in df['primary_artist'].unique():
        artistname = re.sub(r'['+chars+']', '', artist) if artist in ["T'Pau", "Des'ree", "K'NAAN", "Rag'n'Bone Man"] else artist
        clean_artistname = artistname.replace('&', 'and') if '&' in artistname else artistname
        try:  
            results = sp.search(q='artist:' + clean_artistname, type='artist')
            genres = results['artists']['items'][0]['genres']

            artist_idx = df[df['primary_artist'] == str(artist)].index
            for idx in artist_idx:
                df.at[idx, 'genres'] = genres
        except:
            print('error at', artist)
        
get_genre_by_artist(songs_df)

In [45]:
chars = r"""!"\#\$%\&'\(\)\*\+,\\./:;<=>\?@\[\\\]\^_`\{\|\}\~’"""
def get_lyrics_by_song(df):
    for index, row in df.iterrows():
        artist = df.iloc[index]['primary_artist']
        song = df.iloc[index]['title']
        
        if artist == 'Yusuf / Cat Stevens':
            artist = 'Cat Stevens'
            
#         Ö, é, ý, $, ë, !, Ø, ó, í
        artistname = artist.replace('&', 'and') if '&' in artist else artist
        artistname_2 = artistname.replace(' /', '') if ' /' in artistname else artistname
        clean_artistname = re.sub(r'['+chars+']', '', artistname_2)
        artistname_url =  clean_artistname.replace(' ','-') if ' ' in clean_artistname else clean_artistname

        songname = song.replace('&', 'and') if '&' in song else song
        for char in [' / ', '...']:
            if char in songname:
                songname = songname.replace(char, ' ')
                
#         stop_words = ["(Pt. 1", "(Part 1)", "(Part 2)", "(Live)", "- Live", "(feat. ", "(Theme", "- Theme", "(with", "- Remaster",
#                       "- 2", "- 1", "- Ultimate Mix", "- Extended", "(Extended", "- The Original", "- Rerecord", "- Single", 
#                       "- Commercial", "- Original", "- Remixed", "- Remix" " Pt.", "- Love Theme", "- Edit", "- Non", "- From", 
#                       "- Alt.", "- Pts", "- Stereo", "- English", "- Radio", "- Long", "- US", "- Tous", "- '", "- 7"]
        stop_words = ["(Pt.", "(Part", "(Live)", "(feat.", "[feat", "(ft.", "(Theme", "(with", "(From", "(Extended", " Pt.", "- "]
        for string in stop_words:
            if string in songname:
                songname = songname.split(string)[0][:-1]
        songname_2 = re.sub(r'['+chars+']', '', songname)
        clean_songname = songname_2.replace(' ','-') if ' ' in songname_2 else songname_2
        songname_url = clean_songname[:-1] if clean_songname[-1:] == '-' else clean_songname

        url = 'https://genius.com/'+ artistname_url + '-' + songname_url + '-' + 'lyrics'
        page = requests.get(url)
#         json = page.json()
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics_container = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-6 YYrds")

        if lyrics_container: 
            lyrics = lyrics_container.text
        elif lyrics_container == None:
            lyrics = None
            print(df.at[index,'primary_artist']+':', df.at[index,'title'], '||||', url)

        df.at[index, 'lyrics'] = lyrics
        
get_lyrics_by_song(songs_df)

Creedence Clearwater Revival Experience: Travelling Band |||| https://genius.com/Creedence-Clearwater-Revival-Experience-Travelling-Band-lyrics
Marmalade: Reflections of My Life |||| https://genius.com/Marmalade-Reflections-of-My-Life-lyrics
Allman Brothers Band: Midnight Rider |||| https://genius.com/Allman-Brothers-Band-Midnight-Rider-lyrics
T. Rex: Bang a Gong (Get It On) - 2003 Remaster |||| https://genius.com/T-Rex-Bang-a-Gong-Get-It-On-lyrics
Tommy James & The Shondells: Draggin' the Line |||| https://genius.com/Tommy-James-and-The-Shondells-Draggin-the-Line-lyrics
Andy Williams: Where Do I Begin - Love Theme from "Love Story" |||| https://genius.com/Andy-Williams-Where-Do-I-Begin-lyrics
Hamilton, Joe Frank & Reynolds: Don't Pull Your Love Out |||| https://genius.com/Hamilton-Joe-Frank-and-Reynolds-Dont-Pull-Your-Love-Out-lyrics
Nitty Gritty Dirt Band: Mr. Bojangles |||| https://genius.com/Nitty-Gritty-Dirt-Band-Mr-Bojangles-lyrics
Tami Lynn: I'm Gonna Run Away from You |||| http

Sananda Maitreya: Sign Your Name |||| https://genius.com/Sananda-Maitreya-Sign-Your-Name-lyrics
Will To Power: Baby, I Love Your Way/Freebird |||| https://genius.com/Will-To-Power-Baby-I-Love-Your-WayFreebird-lyrics
Enya: Orinoco Flow |||| https://genius.com/Enya-Orinoco-Flow-lyrics
Metallica: One (Remastered) |||| https://genius.com/Metallica-One-Remastered-lyrics
Milli Vanilli: I'm Gonna Miss You |||| https://genius.com/Milli-Vanilli-Im-Gonna-Miss-You-lyrics
Sinéad O'Connor: Nothing Compares 2 U |||| https://genius.com/Sinéad-OConnor-Nothing-Compares-2-U-lyrics
AC/DC: Thunderstruck |||| https://genius.com/ACDC-Thunderstruck-lyrics
C & C Music Factory: Gonna Make You Sweat (Everybody Dance Now) |||| https://genius.com/C-and-C-Music-Factory-Gonna-Make-You-Sweat-Everybody-Dance-Now-lyrics
AC/DC: Moneytalks |||| https://genius.com/ACDC-Moneytalks-lyrics
Go West: The King of Wishful Thinking |||| https://genius.com/Go-West-The-King-of-Wishful-Thinking-lyrics
Tony! Toni! Toné!: Feels Good 

P!nk: Don't Let Me Get Me - Radio Edit |||| https://genius.com/Pnk-Dont-Let-Me-Get-Me-lyrics
Céline Dion: A New Day Has Come - Radio Remix |||| https://genius.com/Céline-Dion-A-New-Day-Has-Come-lyrics
Tweet: Oops (Oh My) [feat. Missy Elliott] |||| https://genius.com/Tweet-Oops-Oh-My-feat-Missy-Elliott-lyrics
Beyoncé: Crazy In Love (feat. Jay-Z) |||| https://genius.com/Beyoncé-Crazy-In-Love-lyrics
Beyoncé: Baby Boy (feat. Sean Paul) |||| https://genius.com/Beyoncé-Baby-Boy-lyrics
Nelly: Shake Ya Tailfeather (feat. P. Diddy & Murphy Lee) - 2016 Remaster |||| https://genius.com/Nelly-Shake-Ya-Tailfeather-lyrics
Mýa: My Love Is Like...Wo - Main Mix |||| https://genius.com/Mýa-My-Love-Is-Like-Wo-lyrics
JAY-Z: Numb / Encore |||| https://genius.com/JAY-Z-Numb-Encore-lyrics
Ciara: One, Two Step (feat. Missy Elliott) |||| https://genius.com/Ciara-One-Two-Step-lyrics
Michael Bublé: Home |||| https://genius.com/Michael-Bublé-Home-lyrics
Sean Paul: We Be Burnin' |||| https://genius.com/Sean-Paul-W

Offset: Ric Flair Drip (with Metro Boomin) |||| https://genius.com/Offset-Ric-Flair-Drip-lyrics
Kendrick Lamar: LOVE. FEAT. ZACARI. |||| https://genius.com/Kendrick-Lamar-LOVE-FEAT-ZACARI-lyrics
Dynoro: In My Mind |||| https://genius.com/Dynoro-In-My-Mind-lyrics
Sofía Reyes: 1, 2, 3 (feat. Jason Derulo & De La Ghetto) |||| https://genius.com/Sofía-Reyes-1-2-3-lyrics
Nio Garcia: Te Boté - Remix |||| https://genius.com/Nio-Garcia-Te-Boté-lyrics
Piso 21: Déjala Que Vuelva (feat. Manuel Turizo) |||| https://genius.com/Piso-21-Déjala-Que-Vuelva-lyrics
Tiësto: Jackie Chan |||| https://genius.com/Tiësto-Jackie-Chan-lyrics
Dan + Shay: Tequila |||| https://genius.com/Dan--Shay-Tequila-lyrics
The Weeknd: Pray For Me (with Kendrick Lamar) |||| https://genius.com/The-Weeknd-Pray-For-Me-lyrics
Maluma: Corazón (feat. Nego do Borel) |||| https://genius.com/Maluma-Corazón-lyrics
Shawn Mendes: Señorita |||| https://genius.com/Shawn-Mendes-Señorita-lyrics
Drake: Money In The Grave (Drake ft. Rick Ross) 

In [49]:
songs_df[songs_df['lyrics'].isnull()]

,id,title,primary_artist,all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_s,year,genres,lyrics
80,5OLHXcOkS53eUHRnADf4ht,Travelling Band,Creedence Clearwater Revival Experience,['Creedence Clearwater Revival Experience'],0.532,0.784,0,-7.067,0,0.0402,0.2100,0.000007,0.4660,0.968,156.976,4,133,1970,[],None
95,3R3s733nwA8Yi1BMk4NGjx,Reflections of My Life,Marmalade,['Marmalade'],0.388,0.406,7,-10.666,1,0.0266,0.5720,0.035100,0.2950,0.510,92.668,4,251,1970,"[bubblegum pop, classic uk pop, merseybeat]",None
113,1hHGUdFaiJHw9hVZGJWYO2,Midnight Rider,Allman Brothers Band,['Allman Brothers Band'],0.585,0.532,7,-9.673,1,0.0268,0.2090,0.007970,0.2270,0.680,94.461,4,180,1971,"[album rock, blues rock, classic rock, country...",None
130,6FsQrRpBLgsrFeAeiQqytm,Bang a Gong (Get It On) - 2003 Remaster,T. Rex,['T. Rex'],0.729,0.818,2,-9.992,1,0.0732,0.1570,0.284000,0.1950,0.906,125.814,4,267,1971,"[album rock, art rock, blues rock, classic roc...",None
135,4WYFQLVqla6tpYtKsSqcgu,Draggin' the Line,Tommy James & The Shondells,['Tommy James & The Shondells'],0.669,0.583,1,-9.165,0,0.0689,0.0521,0.000015,0.1770,0.647,114.548,4,165,1971,"[brill building pop, bubblegum pop, classic ga...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4759,2ksOAxtIxY8yElEWw8RhgK,China,Anuel AA,"['Anuel AA', 'Daddy Yankee', 'KAROL G', 'J Bal...",0.786,0.808,7,-3.702,1,0.0882,0.0846,0.000289,0.0822,0.609,105.027,4,302,2019,"[reggaeton, reggaeton flow, trap boricua, trap...",None
4764,5W83ErFkO3aKAIS1WMi6u0,Secreto,Anuel AA,"['Anuel AA', 'KAROL G']",0.807,0.803,11,-4.156,1,0.1260,0.6020,0.008530,0.1360,0.706,91.987,4,259,2019,"[reggaeton, reggaeton flow, trap boricua, trap...",None
4771,5iwz1NiezX7WWjnCgY5TH4,Calma - Remix,Pedro Capó,"['Pedro Capó', 'Farruko']",0.826,0.773,11,-4.218,0,0.0524,0.3230,0.000000,0.1430,0.761,126.899,4,238,2019,"[latin pop, puerto rican pop]",None
4775,4hPpVbbakQNv8YTHYaOJP4,One Thing Right,Marshmello,"['Marshmello', 'Kane Brown']",0.659,0.625,4,-2.253,1,0.0450,0.0644,0.000000,0.5820,0.442,88.042,4,182,2019,"[brostep, dance pop, edm, pop, pop dance, prog...",None


In [50]:
with open('./data/merge_with_lyrics_and_genres.csv', 'w', encoding='utf-8') as f:
        songs_df.to_csv(f, header= True, index=False)